# Imports

In [1]:
import numpy as np
import torch

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

from torchvision import transforms as T
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter

import pacs

from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset
import numpy as np

# Config

## Regarding Dataset

In [ ]:
NUM_CLASSES = 7
CLASSES = ["dog", "elephant", "giraffe", "guitar", "horse", "house", "person"]
DOMAINS = {"art_painting", "cartoon", "photo", "sketch"}

## Hyperparameters

In [3]:
EPOCHS = 10
BATCH_SIZE = 18
LEARNING_RATE = 0.001 # Standard value for Adam: 0.001
REGULARIZATION = 0 # Standard value for Adam: 0
BETAS = (0.9, 0.999) # Standard values for Adam: (0.9, 0.999)

## Image Normalization

In [4]:
# Values for pretrained ResNet
# image_transform = T.Compose([
#     T.Resize(256),
#     T.CenterCrop(224),
#     T.ToTensor(),
#     T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

## Device

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Device: {device}")

Device: cuda


## Set the Seed for Reproducibility

In [6]:
seed = 42
torch.manual_seed(42)
if device == torch.device("cuda"):
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# Training

In [7]:
writer = SummaryWriter()
%load_ext tensorboard
%tensorboard --logdir ./runs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/hans/anaconda3/envs/xai_proj_t2/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/home/hans/anaconda3/envs/xai_proj_t2/lib/python3.11/site-packages/tensorboard/main.py", line 27, in <module>
    from tensorboard import default
  File "/home/hans/anaconda3/envs/xai_proj_t2/lib/python3.11/site-packages/tensorboard/default.py", line 33, in <module>
    from tensorboard.plugins.core import core_plugin
  File "/home/hans/anaconda3/envs/xai_proj_t2/lib/python3.11/site-packages/tensorboard/plugins/core/core_plugin.py", line 32, in <module>
    from tensorboard.util import grpc_util
  File "/home/hans/anaconda3/envs/xai_proj_t2/lib/python3.11/site-packages/tensorboard/util/grpc_util.py", line 24, in <module>
    import grpc
  File "/home/hans/anaconda3/envs/xai_proj_t2/lib/python3.11/site-packages/grpc/__init__.py", line 22, in <module

## Training Loop

In [8]:
class AverageMeter:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [9]:
def accuracy(target, output):
    batch_size = target.shape[0]
    _, pred = torch.max(output, dim=-1)
    correct = pred.eq(target).sum()
    return correct.item() / batch_size

In [10]:
def train(epoch, target, data_loader, model, optimizer) -> tuple[float, float]:
    """train one epoch"""
    model.train()
    losses = AverageMeter()
    accs = AverageMeter()

    for i, (data, target) in enumerate(data_loader):
        step = (epoch - 1) * len(data_loader) + i + 1
        data = data.to(device)
        target = target.to(device)

        out = model(data)
        loss = F.cross_entropy(out, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = accuracy(target, out)
        losses.update(loss.item(), out.shape[0])
        accs.update(acc, out.shape[0])

        writer.add_scalar(f'Loss/train_{target}', loss.item(), step)
        writer.add_scalar(f'Accuracy/train_{target}', acc, step)

    return losses.avg, accs.avg

## Evaluation

In [11]:
def evaluate(data_loader, model, phase="val") -> tuple[float, float]:
    model.eval()
    losses = AverageMeter()
    accs = AverageMeter()

    with torch.no_grad():
        for data, target in data_loader:
            data = data.to(device)
            target = target.to(device)

            out = model(data)
            loss = F.cross_entropy(out, target)

            acc = accuracy(target, out)
            losses.update(loss.item(), out.shape[0])
            accs.update(acc, out.shape[0])
    
    return losses.avg, accs.avg

In [17]:
k_folds = 5
results = []



for target_domain in tqdm(DOMAINS, desc="Target Domain"):
    # Data Transformations
    img_mean, img_std = pacs.get_nomalization_stats(target_domain)
    print(f"Normalization stats for {target_domain} - Mean: {img_mean}, Std: {img_std}")
    
    transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=img_mean, std=img_std)
    ])

    # Create the data loaders for this fold
    train_loader, test_loader = pacs.get_data_loaders(
        target_domain=target_domain,
        train_batch_size=32,
        transform=transform,
        target_transform=None
    )

    # Initialize model, loss function, and optimizer
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(2048, NUM_CLASSES)
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=BETAS, weight_decay=REGULARIZATION)
    criterion = nn.CrossEntropyLoss()

    # Training Loop
    best_acc = 0
    for epoch in tqdm(range(1, EPOCHS + 1), desc=f"Epoch ({target_domain})"):
        # Set model to training mode
        model.train()
        
        # Training step
        train_loss, train_acc = train(epoch, model, train_loader, optimizer, criterion)
        
        # Set model to evaluation mode
        model.eval()

        # Evaluation step (no gradients needed)
        test_loss, test_acc = evaluate(model, test_loader, criterion)

        # Save the best model
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), f"models/resnet50/best_{target_domain}.pt")

    # Load best model for final evaluation
    model.load_state_dict(torch.load(f"models/resnet50/best_{target_domain}.pt"))
    _, acc = evaluate(model, test_loader, criterion)

    results.append(acc)


# Results Summary
avg_acc = np.mean(results)
worst_case_acc = np.min(results)

print(f"\nCross-Validation Results:\nAverage Accuracy: {avg_acc}\nWorst-case Accuracy: {worst_case_acc}")

Target Domain:   0%|          | 0/4 [00:00<?, ?it/s]

Normalization stats for cartoon - Mean: tensor([0.7512, 0.7332, 0.7101]), Std: tensor([0.1835, 0.1886, 0.2020])


Epoch (cartoon):   0%|          | 0/10 [00:00<?, ?it/s]

AttributeError: 'Adam' object has no attribute 'train'